In [1]:
# !pip install pydrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials


# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)
# # https://drive.google.com/open?id=1tl_Gpc8AOHgMqID4rWK5ZUfOKwxx8e2g
# # https://drive.google.com/open?id=1Dz4nTn_AiVKAXlRXjlI9NhZQzd1X9h4f
# download = drive.CreateFile({'id': '1Dz4nTn_AiVKAXlRXjlI9NhZQzd1X9h4f'})
# download.GetContentFile('parallel.tgz')
# ! tar -xf parallel.tgz

In [1]:
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
import string 
import nltk
# nltk.download('punkt')

In [148]:
file=open("parallel/parallel/IITB.en-hi.en",encoding="utf-8").read()
file2=open("parallel/parallel/IITB.en-hi.hi",encoding="utf-8").read()
eng=file.split('\n')[:1000]
hin=file2.split('\n')[:1000]

In [149]:
x=[]
y=[]
z=0
for i in range(len(eng)):
    if len(eng[i])<1 or len(hin[i])<1:
        continue
    ans1=eng[i].strip()
    ans2=hin[i].strip()
    for j in string.punctuation+"-":
        if j in ans1:
            ans1=ans1.replace(j,"")
        if j in ans2:
            ans2=ans2.replace(j,"")
    x.append(ans1)
    y.append(ans2)


len(x),len(y)

(1000, 1000)

In [150]:

max_l=0
for i in x:
  if(len(i.split())>max_l):
    max_l=len(i.split())
    
max_e=max_l+5
for i in x:
  if(len(i.split())>max_l):
    max_l=len(i.split())
    
max_h=max_l+5
        
mapping_hin={}
demapping_hin={}
mapping_eng={}
demapping_eng={}
k=1
for i in set((" ".join(x)).split()):

    if len(i)<1:
        continue

    if i.lower() in mapping_eng.keys():
        continue
    mapping_eng[i.lower()]=k
    demapping_eng[k]=i.lower()
    k+=1

mapping_eng["startseq"]=k
demapping_eng[k]="startseq"
k+=1
mapping_eng["endseq"]=k
demapping_eng[k]="endseq"
    
k=1
for i in set((" ".join(y)).split()):

    if len(i)<1:
        continue

    if i.lower() in mapping_hin.keys():
        continue
    mapping_hin[i.lower()]=k
    demapping_hin[k]=i.lower()
    k+=1
    
mapping_hin["startseq"]=k
demapping_hin[k]="startseq"
k+=1
mapping_hin["endseq"]=k
demapping_hin[k]="endseq"

def get_batch(s,e):

  data_x=[]
  for i in range(s,e,1):
    k=[]
    k.append(mapping_eng["startseq"])
    for j in x[i].split():
      k.append(mapping_eng[j.lower()])
    k.append(mapping_eng["endseq"])
    k=[0]*(max_e-len(k))+k
    k=np.array(k[:max_e])
    data_x.append(k)
  data_y=[]
  data_t=[]
  for i in range(s,e,1):

    k=[]
    k.append(mapping_hin["startseq"])
    for j in y[i].split():
        k.append(mapping_hin[j.lower()])
  
    k=k+[0]*(max_h-len(k))
    k=np.array(k[:max_h])
    data_y.append(k)

  for i in range(s,e,1):

    k=[]
  
    for j in y[i].split():
        k.append(mapping_hin[j.lower()])
    k.append(mapping_hin["endseq"])
    k=k+[0]*(max_h-len(k))
    k=np.array(k[:max_h])
    data_t.append(k)

  data_x=np.array(data_x)
  data_y=np.array(data_y)
  data_t=np.array(data_t)
  return data_x,data_y,data_t

len(mapping_hin),len(mapping_eng)

(413, 319)

In [151]:
tf.reset_default_graph()

In [6]:
# hyper-parameters
batch_size=1000
nodes=256
embed_size=128



X=tf.placeholder(tf.int32,(None,max_e))
Y=tf.placeholder(tf.int32,(None,max_h))
T=tf.placeholder(tf.int32,(None,max_h))

# embeddings

embeddings_h=tf.Variable(tf.random_uniform((len(mapping_hin)+1,embed_size),-1,1),name="embeddings_h")
embeddings_e=tf.Variable(tf.random_uniform((len(mapping_eng)+1,embed_size),-1,1),name="embeddings_e")
X_embed=tf.nn.embedding_lookup(embeddings_e,X)
Y_embed=tf.nn.embedding_lookup(embeddings_h, Y)


NameError: name 'mapping_hin' is not defined

In [5]:
with tf.variable_scope("encoder"):
    enc_cell=tf.nn.rnn_cell.LSTMCell(nodes,activation=tf.nn.relu,name="enc_cell")
    enc_out,enc_state=tf.nn.dynamic_rnn(enc_cell,X_embed,dtype=tf.float32)

with tf.variable_scope("decoder"):
    dec_cell=tf.nn.rnn_cell.LSTMCell(nodes,activation=tf.nn.relu,name="dec_cell")
    dec_out,_=tf.nn.dynamic_rnn(dec_cell,Y_embed,initial_state=enc_state)

with tf.variable_scope("optimizer"):
    dec_out=tf.layers.dropout(dec_out,0.4)
    logits_h = tf.contrib.layers.fully_connected(dec_out,num_outputs=len(mapping_hin)+1, activation_fn=None)
    loss=tf.contrib.seq2seq.sequence_loss(logits=logits_h,targets=T,weights=tf.ones((batch_size,max_h)))
    optimizer = tf.train.AdamOptimizer().minimize(loss)

NameError: name 'X_embed' is not defined

In [154]:
# sess.close()
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [155]:
#Train
epochs=20


for i in range(epochs):
    k=0
    batch_loss=0

    while k <500:
        batch_loss=0
        batch_logits=0
        acc=0
#         if k+batch_size > data_len:
#             _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],feed_dict={X:data_x[k:],Y:data_y[k:],
#                                                                                         T:data_y[k:]})
#         else:
        data_x,data_y,data_t=get_batch(k,k+batch_size)
#         print(data_x[0],data_y[0],data_t[0])
        _, batch_loss, batch_logits = sess.run([optimizer, loss, logits_h],feed_dict={X:data_x,Y:data_y,T:data_t})
                                                                                            
        accuracy = np.mean(batch_logits.argmax(axis=-1) == data_t[k:k+batch_size])

        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f}'.format(i, batch_loss,accuracy))
        k+=batch_size
        break



Epoch   0 Loss:  6.124 Accuracy: 0.0003
Epoch   1 Loss:  5.501 Accuracy: 0.8371
Epoch   2 Loss:  4.684 Accuracy: 0.8374
Epoch   3 Loss:  2.473 Accuracy: 0.8374
Epoch   4 Loss:  1.966 Accuracy: 0.8372
Epoch   5 Loss: 20.889 Accuracy: 0.8374
Epoch   6 Loss: 101.384 Accuracy: 0.8374
Epoch   7 Loss: 54.025 Accuracy: 0.8374
Epoch   8 Loss: 36.698 Accuracy: 0.8373
Epoch   9 Loss: 34.240 Accuracy: 0.8374
Epoch  10 Loss: 34.270 Accuracy: 0.8374
Epoch  11 Loss: 31.274 Accuracy: 0.8374
Epoch  12 Loss: 23.568 Accuracy: 0.8372
Epoch  13 Loss: 18.029 Accuracy: 0.8374
Epoch  14 Loss: 12.860 Accuracy: 0.8374
Epoch  15 Loss:  9.870 Accuracy: 0.8374
Epoch  16 Loss:  7.873 Accuracy: 0.8374
Epoch  17 Loss:  6.762 Accuracy: 0.8374
Epoch  18 Loss:  5.990 Accuracy: 0.8374
Epoch  19 Loss:  5.513 Accuracy: 0.8374


In [136]:
 def get_reply(text):
    tokens=text.split()
    data=[]
    data.append(mapping_eng["startseq"])
    for i in tokens:
        data.append(mapping_eng[i.lower()])
    data.append(mapping_eng["endseq"])
    data_x=np.array([0]*(max_e-len(data))+data)
    data_y=[]
    data_y=np.array([mapping_hin["startseq"]]+[0]*(max_h-len(data_y)-1))
    last_t=""
    
    k=0
    reply=[]
    
    while(last_t!="endseq" and k<max_h-1):
        
        reply.append(sess.run(logits_h,feed_dict={X:[data_x],Y:[data_y]}).argmax(axis=-1)[0,k])
#         print(data_x)
        data_y[k+1]=reply[k]
#         print(data_y)
        if(reply[k]==0):
            break
        last_t=demapping_hin[reply[k]]
        k+=1
    text=""
#     print(reply)
    for i in reply:
#       print(i)
      if i==0 or i==mapping_hin["endseq"]:
          break
      text=text+" "+demapping_hin[i]
    return text

In [146]:
k=np.random.randint(0,10)
# k=1
get_reply(x[k]),x[k],y[k]

(' अवधि को हाइलाइट रकें', 'Highlight duration', 'अवधि को हाइलाइट रकें')

In [52]:
a,b,c=get_batch(330,331)
sess.run(logits_h,feed_dict={X:a,Y:b})


array([[[ 1.3659948 , -0.6210733 ,  0.57359606, ..., -0.37711018,
          0.00927887,  0.6345121 ],
        [ 1.4139137 , -0.6146766 ,  0.5130726 , ..., -0.4481693 ,
         -0.13215625,  0.6048555 ],
        [ 1.328395  , -0.50415367,  0.48230973, ..., -0.45814547,
         -0.16187964,  0.68956536],
        ...,
        [ 3.7565799 , -1.152199  ,  0.64777684, ..., -1.0389801 ,
         -1.2760469 ,  1.3248634 ],
        [ 4.0230546 , -1.073932  ,  0.6522169 , ..., -1.2420927 ,
         -1.358303  ,  1.5296084 ],
        [ 4.3087034 , -0.9515295 ,  0.6524049 , ..., -1.4894807 ,
         -1.434341  ,  1.7592818 ]]], dtype=float32)

In [ ]:
get_batch(0,1)

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

model.save("facial_model_vgg_cv.h5")
# get the folder id where you want to save your file
file = drive.CreateFile()
file.SetContentFile("facial_model_vgg_cv.h5")
file.Upload()